# GeoMap

https://data.opendatasoft.com/explore/dataset/european-union-countries@public/export/

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json

In [ ]:
import plotly.io as pio
pio.renderers.default = 'browser'

In [ ]:
geoplot = pd.read_csv("food_supply_original.csv")
european_union = json.load(open("european-union-countries.geojson", "r"))

In [ ]:
# check properties
# european_union["features"][1]["properties"]

In [ ]:
smo = pd.read_csv("share-of-adults-who-smoke.csv")
#retrieved from https://ourworldindata.org/smoking
smo = smo[smo["Year"] == 2016].rename(columns={"Code":"iso_a3", "Smoking prevalence, total (ages 15+)":"Smoking"})
smo = smo[["iso_a3", "Smoking"]]
smo.head()

## Preprocessing

In [ ]:
state_id_map = {}
for feature in european_union["features"]:
    feature["id"] = feature["properties"]["gu_a3"]
    state_id_map[feature["properties"]["name_long"]] = feature["id"]

In [ ]:
# create dict with necessary information 
state_id_map

In [ ]:
# rename column and value in column cause doesn't match 
geoplot.rename(columns={"Unnamed: 0": "Country"}, inplace=True)
geoplot["Country"].replace({"Czechia": "Czech Republic"}, inplace=True)

In [ ]:
# create new column with polygon info
geoplot["iso_a3"] = geoplot["Country"].apply(lambda x: state_id_map[x])
geoplot.head()

In [ ]:
geoplot = geoplot.merge(smo, on="iso_a3")

In [ ]:
geoplot = geoplot[['Country', 'Milk - Excluding Butter', 'Oilcrops', 'Eggs',
       'Starchy Roots', 'Meat', 'Vegetables', 'Fruits - Excluding Wine',
       'Aquatic Products, Other', 'Treenuts', 'Spices', 'Pulses',
       'Alcoholic Beverages', 'Cereals - Excluding Beer', 'Animal fats',
       'Vegetable Oils', 'Offals', 'Sugar & Sweeteners', 'Fish, Seafood',
       'Stimulants', 'Smoking', 'Obesity', 'total_cases_per_million',
       'total_deaths_per_million', 'life_expectancy',
       'human_development_index', 'population_density', 'median_age',
       'gdp_per_capita', 'cardiovasc_death_rate',
       'Alcoholic Beverages above avg', 'Vegetables above avg',
       'Obesity above avg', 'iso_a3']]


In [ ]:
geoplot.to_csv("food_supply.csv")

## Plot

In [ ]:
fig = px.choropleth_mapbox(
    geoplot,
    locations="iso_a3",
    geojson=european_union,
    color="Smoking",
    hover_name="Country",
    hover_data=["cardiovasc_death_rate"],
    #title="Life expectancy Europe",
    mapbox_style="carto-positron", # kann hier auch darkmode u.a. - mit api token noch mehr aber to much
    center={"lat": 56.5, "lon": 11},
    zoom=2.5,
    opacity=0.5, # können das hier weniger transparent machen
    height=600,
    width=550
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

## For Dash

Nicht ganz dasselbe weil kein token aber bestimmt übetragbar: 
https://plotly.com/python/mapbox-county-choropleth/

Filter mit variablen dann in dash welche wir für das coloring wollen.